# Imprts


In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


#Afficher toutes le colonnes du data frame : 
pd.set_option('display.max_columns', 200)

# Mes fonction - Rapport Data Frame
from src.mes_fonction import RapportDataFrame

# Data Ingestion : import des data et autres fonctionnalité
from src.components.data_ingestion import *

In [1]:
import os
import sys

# Les données

In [20]:
# Extraction nom des fichiers et noms des data frames
directory_data = DirectoryDataPath()
liste_name, files_liste_name = directory_data.get_files_names()

In [21]:
# La liste des fichiers : ici j'ai enlevé HomeCredit_columns_description car n'arrive pas à le lire avec pd.read_csv()

files_liste_name = ['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 #'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

In [22]:
# Attribution des noms de dataframe respectifs : 
liste_name = ['application_test',
 'application_train',
 'bureau',
 'bureau_balance',
 'credit_card_balance',
 #'HomeCredit_columns_description',
 'installments_payments',
 'POS_CASH_balance',
 'previous_application',
 'sample_submission']

 # Importation et nommage des data sets
path = 'C:/Users/Lenovo/Documents/DSPython/data_projet_7/'
for i, name in enumerate(liste_name):
    exec(f"{name} = pd.read_csv(path + files_liste_name[i])")

Pour importer que les deux sets application_train et application_test, on peut utiliser la méthode suivante : 

In [26]:
data = ImportData()
train__, test__ = data.train_test_set()

## Exploration des données

In [25]:

# Liste des données à afficher : 
summary_list = []
for i, name in enumerate(liste_name):
    df = globals()[name]
    rows, cols = df.shape
    summary = {'DataFrame': name, 'Lignes': rows, 'Colonnes': cols}
    summary_list.append(summary)
summary_df = pd.DataFrame(summary_list)
summary_df

,DataFrame,Lignes,Colonnes
0,application_test,48744,121
1,application_train,307511,122
2,bureau,1716428,17
3,bureau_balance,27299925,3
4,credit_card_balance,3840312,23
5,installments_payments,13605401,8
6,POS_CASH_balance,10001358,8
7,previous_application,1670214,37
8,sample_submission,48744,2


### Rapports data frames

In [29]:
rapport_df_train = RapportDataFrame(application_train)
# Rapport : application_train
rapport_df_train.rapport()

Le Taux de remplissage total est égal à : 75.60405809287056 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******

NAME_CONTRACT_TYPE-------------------------------- 2
CODE_GENDER--------------------------------------- 3
FLAG_OWN_CAR-------------------------------------- 2
FLAG_OWN_REALTY----------------------------------- 2
NAME_TYPE_SUITE----------------------------------- 8
NAME_INCOME_TYPE---------------------------------- 8
NAME_EDUCATION_TYPE------------------------------- 5
NAME_FAMILY_STATUS-------------------------------- 6
NAME_HOUSING_TYPE--------------------------------- 6
OCCUPATION_TYPE----------------------------------- 19
WEEKDAY_APPR_PROCESS_START------------------------ 7
ORGANIZATION_TYPE--------------------------------- 58
FONDKAPREMONT_MODE-------------------------------- 5
HOUSETYPE_MODE------------------------------------ 4
WALLSMATERIAL_MODE-------------------------------- 8
EMER

In [30]:
rapport_df_test = RapportDataFrame(application_test)
# Rapport : application_test
rapport_df_test.rapport()

Le Taux de remplissage total est égal à : 76.18831323846766 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******

NAME_CONTRACT_TYPE-------------------------------- 2
CODE_GENDER--------------------------------------- 2
FLAG_OWN_CAR-------------------------------------- 2
FLAG_OWN_REALTY----------------------------------- 2
NAME_TYPE_SUITE----------------------------------- 8
NAME_INCOME_TYPE---------------------------------- 7
NAME_EDUCATION_TYPE------------------------------- 5
NAME_FAMILY_STATUS-------------------------------- 5
NAME_HOUSING_TYPE--------------------------------- 6
OCCUPATION_TYPE----------------------------------- 19
WEEKDAY_APPR_PROCESS_START------------------------ 7
ORGANIZATION_TYPE--------------------------------- 58
FONDKAPREMONT_MODE-------------------------------- 5
HOUSETYPE_MODE------------------------------------ 4
WALLSMATERIAL_MODE-------------------------------- 8
EMER

In [31]:
rapport_df_bureau = RapportDataFrame(bureau)
# Rapport : bureau
rapport_df_bureau.rapport()

Le Taux de remplissage total est égal à : 86.49744770912068 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******

CREDIT_ACTIVE------------------------------------- 4
CREDIT_CURRENCY----------------------------------- 4
CREDIT_TYPE--------------------------------------- 15


In [32]:
rapport_df_bureau_balance = RapportDataFrame(bureau_balance)
# Rapport : bureau_balance
rapport_df_bureau_balance.rapport()

Le Taux de remplissage total est égal à : 100.0 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******

STATUS-------------------------------------------- 8


In [33]:
rapport_df_bureau_credit_card_balance = RapportDataFrame(credit_card_balance)
# Rapport : credit_card_balance
rapport_df_bureau_credit_card_balance.rapport()

Le Taux de remplissage total est égal à : 93.34592560731252 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******

NAME_CONTRACT_STATUS------------------------------ 7


In [34]:
rapport_df_installments_payments = RapportDataFrame(installments_payments)
# Rapport : installments_payments
rapport_df_installments_payments.rapport()

Le Taux de remplissage total est égal à : 99.99466204634469 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******



In [35]:
rapport_df_POS_CASH_balance = RapportDataFrame(POS_CASH_balance)
# Rapport : POS_CASH_balance
rapport_df_POS_CASH_balance.rapport()

Le Taux de remplissage total est égal à : 99.93481135261831 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******

NAME_CONTRACT_STATUS------------------------------ 9


In [36]:
rapport_df_previous_application = RapportDataFrame(previous_application)
# Rapport : previous_application
rapport_df_previous_application.rapport()

Le Taux de remplissage total est égal à : 82.02312252655503 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******

NAME_CONTRACT_TYPE-------------------------------- 4
WEEKDAY_APPR_PROCESS_START------------------------ 7
FLAG_LAST_APPL_PER_CONTRACT----------------------- 2
NAME_CASH_LOAN_PURPOSE---------------------------- 25
NAME_CONTRACT_STATUS------------------------------ 4
NAME_PAYMENT_TYPE--------------------------------- 4
CODE_REJECT_REASON-------------------------------- 9
NAME_TYPE_SUITE----------------------------------- 8
NAME_CLIENT_TYPE---------------------------------- 4
NAME_GOODS_CATEGORY------------------------------- 28
NAME_PORTFOLIO------------------------------------ 5
NAME_PRODUCT_TYPE--------------------------------- 3
CHANNEL_TYPE-------------------------------------- 8
NAME_SELLER_INDUSTRY------------------------------ 11
NAME_YIELD_GROUP---------------------------------- 5
PRO

In [37]:
rapport_df_sample_submission = RapportDataFrame(sample_submission)
# Rapport : previous_application
rapport_df_sample_submission.rapport()

Le Taux de remplissage total est égal à : 100.0 %
Le Nombre de features vides est égal à : 0 Features
Les Features vides sont : []

*****Nombre de catégorie features catégorielles******



In [20]:
original_columns, categorical_columns, numerical_columns = rapport_df_.get_df_columns()

In [21]:
original_columns

['SK_ID_CURR',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEAR

In [22]:
categorical_columns

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [23]:
numerical_columns

['SK_ID_CURR',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 